Importacion libreria necesarias:

In [1]:
%pip install --upgrade google-cloud-aiplatform
%pip install --quiet google-cloud-storage pillow matplotlib
%pip install -U langchain
%pip install --quiet -U langchain-google-vertexai langchain_core langchain_community tavily-python
%pip install --quiet -U langchain_core langgraph
%pip install google-cloud-aiplatform pydantic
%pip install google-cloud-bigquery-storage
%pip install --quiet python-dotenv
%pip install --quiet pandas as pd
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests











[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgr

In [2]:
#Solo Usar si es necesario borrar credenciales antiguas:
import os
if "GOOGLE_APPLICATION_CREDENTIALS" in os.environ:
    del os.environ["GOOGLE_APPLICATION_CREDENTIALS"]


In [2]:
#Solo Usar si es necesario crear credenciales de GCP
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=QxnCxm5OcjSavcygTHoa1TMtAfwfjJ&access_type=offline&code_challenge=rlAWxmYA--d0NMxoDVh18_MnSELLsIjnbaQLSQSMU1Q&code_challenge_method=S256


Credentials saved to file: [/Users/andresrsalamanca/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "thecarmentor-mvp2" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [4]:
from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Acceso a las variables
google_credentials = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
langchain_tracing = os.getenv("LANGSMITH_TRACING")
langchain_endpoint = os.getenv("LANGSMITH_ENDPOINT")
langchain_api_key = os.getenv("LANGSMITH_API_KEY")
langchain_project = os.getenv("LANGSMITH_PROJECT")

# Imprimir valores para confirmar (solo para depuración)
print(f"Google Credentials: {google_credentials}")
print(f"LangChain Tracing: {langchain_tracing}")
print(f"LangChain Endpoint: {langchain_endpoint}")
print(f"LangChain API Key: {langchain_api_key}")
print(f"LangChain Project: {langchain_project}")


Google Credentials: /Users/andresrsalamanca/.config/gcloud/application_default_credentials.json
LangChain Tracing: true
LangChain Endpoint: https://eu.api.smith.langchain.com
LangChain API Key: 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
LangChain Project: mvp_carmentor


### Autenticacion GCP:

In [2]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=dxUZAjvSpevQ3KgNsrYXQaE4CbkMRp&access_type=offline&code_challenge=q1vQXOdha4ieGRZDcs6m7HTEv5l1-2MMdLjLM9i2VRo&code_challenge_method=S256


Credentials saved to file: [/Users/andresrsalamanca/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "car-agent-444008" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


### Configuracion

1.	Configuración del mensaje de sistema

In [6]:
import pandas as pd
import google.cloud.bigquery as bigquery
from langchain_google_vertexai import ChatVertexAI
from langchain import agents
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from vertexai.preview import reasoning_engines
from langchain_core.messages import HumanMessage, SystemMessage , AIMessage , BaseMessage


# Configurar el cliente de BigQuery
client = bigquery.Client(project="thecarmentor-mvp2")

# Configuración del modelo VertexAI
model = "gemini-1.5-flash-001"  # # Gemini 1.5 Flash / gemini-1.5-pro-002 / gemini-1.5-flash-00
model_kwargs = {
    "project": "thecarmentor-mvp2",
    "location": "europe-west1",
    "temperature": 0.2,
    "max_output_tokens": 240,
}

# Mensaje del sistema para el asistente ""
system_instruction = """""Tu nombre es Mentor, un util asistente para la busqueda de coches.
INSTRUCCIONES IMPORTANTES:
- Cuando realices una búsqueda en la base de datos, SIEMPRE debes incluir los DETALLES COMPLETOS de los coches encontrados.
- Formato de respuesta: 
  * Nombre del coche
  * Precio
  * Kilómetros
  * Mecánica
  * Distancia de similitud
- Si encuentras coches, muestra OBLIGATORIAMENTE la información detallada
- Si no hay resultados, indica claramente que no se encontraron coches"""


In [ ]:
# # Prueba de conexion haciendo una query al modelo Gemini
# agent = reasoning_engines.LangchainAgent(
#     model=model,                # Required.
#     model_kwargs=model_kwargs,  # Optional.
# )

# response = agent.query(input="what is AI Agentic?")
# print(response)

{'input': 'what is AI Agentic?', 'output': '"AI Agentic" is not a widely recognized or established term in the field of artificial intelligence. It\'s possible that it\'s a newly coined term or a specific concept within a particular research area. \n\nTo understand what "AI Agentic" might mean, we need more context. Could you please provide more information about where you encountered this term? For example:\n\n* **What was the source of the information?** (e.g., a research paper, a blog post, a conversation)\n* **What was the context of the discussion?** (e.g., discussing AI ethics, AI capabilities, AI design)\n* **What were the key ideas or concepts being discussed?**\n\nWith more information, I can help you understand the meaning of "AI Agentic" and its potential implications. \n'}


### Configuracion Tool (Consulta base de embeddings)

In [8]:
@tool
def buscar_producto_bd(consulta: str):
    """
    Busca productos en la base de datos utilizando una consulta semántica en BigQuery\
    Tu objetivo es proporcionar respuestas precisas para ayudar en la búsqueda en el inventarios de coches disponibles.
    Args:
        consulta (str): Consulta de texto para buscar productos similares.

    Returns:
        DataFrame: Resultados de la consulta en BigQuery con los detalles de los productos más relevantes.
    """
    try:
        # Construcción de la consulta SQL
        query = f"""
        SELECT 
            base.content AS nombre_coche,
            base.mecanica,
            base.price,
            base.KM,
            search_result.distance
        FROM VECTOR_SEARCH(
            TABLE `web_cars.coches_embeddingsV1`,
            'ml_generate_embedding_result',
            (SELECT * FROM ML.GENERATE_EMBEDDING(
                MODEL `thecarmentor-mvp2.mymodel.modelembedding`,
                (SELECT '{consulta}' AS content),
                STRUCT(TRUE AS flatten_json_output, 'SEMANTIC_SIMILARITY' AS task_type, 768 AS output_dimensionality)
            )),
            'ml_generate_embedding_result',
            top_k => 5
        ) AS search_result
        """

        # Ejecutar la consulta
        query_job = client.query(query)
        results = query_job.result().to_dataframe()

        # Formatear los resultados para el modelo
        formatted_results = []
        for _, row in results.iterrows():
            formatted_results.append(
                f"Coche: {row['nombre_coche']}, "
                f"Mecánica: {row['mecanica']}, "
                f"Precio: {row['price']}€, "
                f"Kilómetros: {row['KM']}km, "
                f"Similitud: {row['distance']:.2f}"
            )
        
        return "\n".join(formatted_results)

    except Exception as e:
        print(f"Error al ejecutar la búsqueda: {e}")
        return "No se pudieron encontrar resultados."
# Definir herramientas
tools = [buscar_producto_bd]

In [9]:
# Query de consulta de prueba con Vector Search
query = """
SELECT 
  base.content,
  base.mecanica,
  base.price,
  base.KM,
  distance
FROM VECTOR_SEARCH(
  TABLE `web_cars.coches_embeddingsV1`,
  'ml_generate_embedding_result',
  (SELECT * FROM ML.GENERATE_EMBEDDING(
     MODEL `thecarmentor-mvp2.mymodel.modelembedding`,
     (SELECT 'mazda 3 diesel' AS content),
     STRUCT(TRUE AS flatten_json_output, 'SEMANTIC_SIMILARITY' AS task_type, 768 AS output_dimensionality)
  )),
  'ml_generate_embedding_result',
  top_k => 5
);
"""

# Ejecuta la consulta y convierte los resultados en un DataFrame de Pandas
query_job = client.query(query)
results = query_job.result().to_dataframe()

# Muestra el DataFrame
results.head()

,content,mecanica,price,KM,distance
0,mazda mazda3 1.6 crtd mirai | 2013 | diesel | ...,diesel,7800,138000,0.159085
1,mazda mazda3 2.2 de 150 mt luxury safety sdn |...,diesel,14900,115000,0.169148
2,mazda cx 3 1.5 skyactiv de style 2wd | 2016 | ...,diesel,14000,89000,0.183444
3,mazda cx 3 1.5 skyactiv de luxury 2wd | 2018 |...,diesel,16290,124906,0.189991
4,mazda cx 3 1.5 skyactiv de luxury 2wd | 2018 |...,diesel,19200,68763,0.190613


### Cracion de Agente

In [10]:
agent = reasoning_engines.LangchainAgent(
    model=model,                # Required.
    tools=tools,
    model_kwargs=model_kwargs,  # Optional.
    system_instruction=system_instruction,
    )

In [11]:
# Probar el agente
response = agent.query(input="mazda 3 diesel")


In [12]:
import pprint
pp = pprint.PrettyPrinter(indent=3)
pp.pprint(response)

{  'input': 'mazda 3 diesel',
   'output': '- **Coche:** mazda mazda3 1.6 crtd mirai\n'
             '- **Precio:** 7800 €\n'
             '- **Kilómetros:** 138000 km\n'
             '- **Mecánica:** diesel\n'
             '- **Distancia de similitud:** 0.16\n'
             '- **Coche:** mazda mazda3 2.2 de 150 mt luxury safety sdn\n'
             '- **Precio:** 14900 €\n'
             '- **Kilómetros:** 115000 km\n'
             '- **Mecánica:** diesel\n'
             '- **Distancia de similitud:** 0.17\n'
             '- **Coche:** mazda cx 3 1.5 skyactiv de style 2wd\n'
             '- **Precio:** 14000 €\n'
             '- **Kilómetros:** 89000 km\n'
             '- **Mecánica:** diesel\n'
             '- **Distancia de similitud:** 0.18\n'
             '- **Coche:** mazda cx 3 1.5 skyactiv de luxury 2wd\n'
             '- **Precio:** 16290 €\n'
             '- **Kilómetros:** 1249'}
